To publish parliament data as parlamint corpus we neet to tag speeches with speaker. This needs to be extracted

In [168]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
from my_functions import rtojson
from bs4.element import Tag

In [169]:
# lenker
sesjoner = "https://data.stortinget.no/eksport/sesjoner"
publikasjoner = "https://data.stortinget.no/eksport/publikasjoner?publikasjontype=referat&sesjonid={}"
referat = "https://data.stortinget.no/eksport/publikasjon?publikasjonid={}"

In [170]:
def get_ref(
    id,
    url = "https://data.stortinget.no/eksport/publikasjon?publikasjonid={}"
):
    r = requests.get(url.format(id))
    soup = bs(r.content, "lxml-xml")
    return soup

In [171]:
r = requests.get(sesjoner)
soup = bs(r.content, "lxml")
s = [x.find('id').text for x in soup.find_all('sesjon')]

Publications from 2020

In [172]:
r = requests.get(publikasjoner.format(s[1]))
soup = bs(r.content, "lxml")
pubs = [x.text for x in soup.find_all('id')]

Publications from 1998-99

In [173]:
r = requests.get(publikasjoner.format('1999-2000'))
soup = bs(r.content, "lxml")
pubs99 = [x.text for x in soup.find_all('id')]

In [175]:
pubs99[-1]

's991221'

In [23]:
doc = get_ref(pubs[1])

In [25]:
doc.find("President")

<President Id="i1001447" personID="ANNH">
<A Id="refe-202021-03-10-7">Møteleder: <Uth Id="refe-202021-03-10-8" Type="Sperret">Anniken Huitfeldt</Uth> 
(utenriks- og forsvarskomiteens leder)</A>
</President>

In [27]:
lst = []
for pub in pubs:
    d = get_ref(pub)
    lst += [d.find("President")]

In [38]:
doc = get_ref(pubs[0])

In [46]:
mote = doc.find('Mote')

In [69]:
Tag

bs4.element.Tag

In [70]:
isinstance(mote, Tag)

True

In [93]:
for child in mote.children:
    if isinstance(child, Tag):
        print('1: ', child.name)
        for child1 in child:
            if isinstance(child1, Tag):
                #print('2: ', child1.name)
                for child2 in child1.children:
                    if isinstance(child2, Tag):
                        print('2: ', child2.name)
                        for child3 in child2:
                            print(child.name)
                            print("Child: ", child3)
                            print(type(child))
        break

1:  Startseksjon
2:  Tittel
Startseksjon
Child:  Møte torsdag den 4. februar 2021
kl. 10
<class 'bs4.element.Tag'>
2:  President
Startseksjon
Child:  

<class 'bs4.element.Tag'>
Startseksjon
Child:  <A Id="refs-202021-02-04-7">President: <Uth Id="refs-202021-02-04-8" Type="Sperret">Magne Rommetveit</Uth>
</A>
<class 'bs4.element.Tag'>
Startseksjon
Child:  

<class 'bs4.element.Tag'>
2:  Dagsorden
Startseksjon
Child:  

<class 'bs4.element.Tag'>
Startseksjon
Child:  <Tittel Id="refs-202021-02-04-10">Dagsorden <DagsordenNR>(nr.
46):</DagsordenNR>
</Tittel>
<class 'bs4.element.Tag'>
Startseksjon
Child:  

<class 'bs4.element.Tag'>
Startseksjon
Child:  <DagsordenSak Id="i1012775">
<Liste Id="refs-202021-02-04-11" Type="Dagsorden">
<Pkt Id="refs-202021-02-04-12">
<A Id="refs-202021-02-04-13">1.	Innstilling
fra helse- og omsorgskomiteen om Midlertidige endringer i smittevernloven
(oppholdssted under innreisekarantene mv.)</A>
<A Id="refs-202021-02-04-14">(Innst. 184 L (2020–2021), jf. Prop.


In [176]:
def to_txt(xml):
    sak = None    
    for tag in xml.descendants:
        if tag.name == 'Saktittel':
            yield('Saktittel')
            yield(tag.text) 
            sak = tag.text   
        elif tag.name == "Presinnlegg":
            yield(tag.name)
            yield(tag.text)
        elif tag.name == "Hovedinnlegg" or tag.name == 'Replikk':
            navn = tag.find('Navn')
            #print(navn['personID'], tag.name)
            yield(tag.name)
            #yield("Sak: ", sak)
            yield(navn.text)
            yield(tag.text.replace(navn.text, '').lstrip())

In [129]:
print(pubs[-1])

refs-202021-12-19


OBS få med merknader!

In [132]:
d = get_ref(pubs[-1])

In [135]:
with open("source.txt", "w") as f:
    f.write(d.text)

In [145]:
with open("cleaned.txt", "w") as f:
    for line in to_txt(get_ref(pubs[-1])):
        f.writelines(line)

In [119]:
sak = None

for tag in mote.descendants:
    if tag.name == 'Saktittel':
        print('Saktittel')
        print(tag.text) 
        sak = tag.text   
    elif tag.name == "Presinnlegg":
        print(tag.name)
        print(tag.text)
    elif tag.name == "Hovedinnlegg" or tag.name == 'Replikk':
        navn = tag.find('Navn')
        print(navn['personID'], tag.name)
        print("Sak: ", sak)
        print(navn.text)
        print(tag.text.replace(navn.text, '').lstrip())

Presinnlegg


Presidenten: Presidenten
vil av smittevernomsyn tilrå at publikumsgalleriet vert halde stengt
også under møtet i dag. – Det er vedteke.
Frå representanten
Camilla Strandskog ligg
det føre søknad om at den godkjende foreldrepermisjonen vert utvida
til å gjelda til og med 26. februar. 
Etter forslag frå
presidenten vart samrøystes vedteke:


Søknaden
vert handsama straks og innvilga.


Vararepresentanten,
Mats A. Kirkebirkeland,
held fram å møta i Stortinget til og med 25. februar.



Presinnlegg


Presidenten: Representanten
Siv Mossleth vil setja fram eit representantforslag. 

SIMOS Hovedinnlegg
Sak:  None

Siv Mossleth (Sp) [10:01:14]:
Jeg er glad for å kunne fremme
forslag fra stortingsrepresentantene Marit Arnstad, Aud Hove, Willfred
Nordlund og meg selv om reetablering av høyere utdanning på Nesna.

Presinnlegg


Presidenten: Representanten
Kari Elisabeth Kaski vil setja fram eit representantforslag.

KEKA Hovedinnlegg
Sak:  None

Kari Elisabeth Kaski (SV) [10:01:47]

In [54]:
type(mote)

bs4.element.Tag

In [44]:
for p in doc.find_all('Navn'):
    print(p.parent)

<A Id="refs-202021-02-04-42">
<Navn Id="refs-202021-02-04-43">Presidenten:</Navn> Presidenten
vil av smittevernomsyn tilrå at publikumsgalleriet vert halde stengt
også under møtet i dag. – Det er vedteke.</A>
<A Id="refs-202021-02-04-54">
<Navn Id="refs-202021-02-04-55">Presidenten:</Navn> Representanten
Siv Mossleth vil setja fram eit representantforslag. </A>
<A Id="refs-202021-02-04-56">
<Navn Id="refs-202021-02-04-57" personID="SIMOS">
Siv Mossleth (Sp) [10:01:14]:</Navn> Jeg er glad for å kunne fremme
forslag fra stortingsrepresentantene Marit Arnstad, Aud Hove, Willfred
Nordlund og meg selv om reetablering av høyere utdanning på Nesna.</A>
<A Id="refs-202021-02-04-59">
<Navn Id="refs-202021-02-04-60">Presidenten:</Navn> Representanten
Kari Elisabeth Kaski vil setja fram eit representantforslag.</A>
<A Id="refs-202021-02-04-61">
<Navn Id="refs-202021-02-04-62" personID="KEKA">
Kari Elisabeth Kaski (SV) [10:01:47]:</Navn> På vegne av representanten
Lars Haltbrekken og meg selv vil 

In [167]:
pub99

NameError: name 'pub99' is not defined

# Publications 99-00

In [179]:
for pub in pubs99[:2]:
    with open(f"proc_txt/{pub}.txt", "w") as f:
        for line in to_txt(get_ref(pub)):
            f.writelines(line)

In [181]:
"\n".join(to_txt(get_ref(pubs99[-1])))

''

In [182]:
pubs99[-1]

's991221'

In [183]:
get_ref(pubs99[-1])

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE forhandling PUBLIC "-//Stortingets EDB-seksjon//DTD//XML Forhandling 2.0//NO" "http://www.stortinget.no/dtd/forhandl_xml.dtd">
<forhandling>
<mote>
<formalia>
<dato>Møte tirsdag den 21.
desember kl. 10</dato>
<president>President: <uth type="sperret">Kirsti Kolle Grøndahl</uth></president>
<dagsorden>
<tit>Dagsorden </tit>
<dagsnr>(nr. 34):</dagsnr>
<ordennr>1.</ordennr>
<saktit>Statsministerens redegjørelse om grunnlaget for å avbryte
arbeidet med å slå sammen Telenor AS og Telia AB</saktit>
<ordennr>2.</ordennr>
<saktit>Innstilling fra forsvarskomiteen om endringer på statsbudsjettet
for 1999 under Forsvarsdepartementet<kildedok>(Innst. S. nr. 84
(1999-2000), jf. St.prp. nr. 6 (1999-2000))</kildedok></saktit>
<ordennr>3.</ordennr>
<saktit>Innstilling fra finanskomiteen om forslag fra stortingsrepresentant
Siv Jensen om endring av regelverk tilknyttet skattelovens § 76
om særfradrag for forsørgelse<kildedok>(Innst.
S. nr. 77 (1999-2000)